In [28]:
import json
import random
import pickle
import os
import sys
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')
import keras
sys.stderr = stderr
from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('Dataset_Creation.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [29]:

import tensorflow as tf
import nltk 
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import SGD
import random
import warnings
warnings.filterwarnings('ignore')

In [30]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
    

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    if tag=='Report':
        return '';
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['response'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    if ints == []:
        res = "Sorry I can't understand kindly rephrase your sentence :)" 
        return res
    #print(ints)
    res = getResponse(ints, intents)
    return res

In [31]:


#print("Please feel free to ask me any thing")
#while(True):
    
 #   s = input("")
  #  if s == "bye":
   #     print(chatbot_response(s))
    #    break
    #print(chatbot_response(s))


In [34]:
import tkinter
from tkinter import *

processing_report=False
count =0
hos_id = None
opd_id = None
phone_no = None

import pandas as pd
report_data = pd.read_csv('Book1.csv').values

def send():
    global processing_report
    global hos_id
    global opd_id
    global phone_no
    global report_data
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        if not processing_report:
            res = chatbot_response(msg)
            if res=='':
                processing_report=True
                res = 'Enter hospital id:'
        else:
            if hos_id==None:
                hos_id = msg
                res = 'Enter opd id:'
            elif opd_id==None:
                opd_id = msg
                res = 'Enter phone number: '
            else:
                phone_no = msg
                found = False
                for data in report_data:
                    if data[0]==hos_id and data[1]==opd_id and data[2]==int(phone_no):
                        res = data[3]+'\n'+data[4]
                        found = True
                if not found:
                    res = 'No Result Found'
                processing_report=False
            
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("COVMedic")
base.geometry("400x480")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=426)
ChatLog.place(x=6,y=6, height=426, width=370)
EntryBox.place(x=6, y=441, height=30, width=295)
SendButton.place(x=310, y=441, height=30, width=80)

base.mainloop()